In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot

from tqdm.notebook import tqdm

import pandas as pd


# Loading File

In [ ]:
# from data_files/CombinedFunctions_from_Fortran/CombinedTypes.h

# from https://www.star.bnl.gov/public/comp/simu/newsite/gstar/Manual/particle_id.html
# (Particle, Options, Mass, Charge, Lifetime)
particles = {
    1: ("gamma", 1, 0.0000E+00, 0., 0.10000E+16),
    2: ("positron", 2, 0.5110E-03, 1., 0.10000E+16),
    3: ("electron", 2, 0.5110E-03, -1., 0.10000E+16),
    4: ("neutrino", 3, 0.0000E+00, 0., 0.10000E+16),
    5: ("mu+", 5, 0.1057E+00, 1., 0.21970E-05),
    6: ("mu-", 5, 0.1057E+00, -1., 0.21970E-05),
    7: ("pi0", 3, 0.1350E+00, 0., 0.84000E-16),
    8: ("pi+", 4, 0.1396E+00, 1., 0.26030E-07),
    9: ("pi-", 4, 0.1396E+00, -1., 0.26030E-07),
    10: ("K0L", 3, 0.4977E+00, 0., 0.51700E-07),
    11: ("K+", 4, 0.4937E+00, 1., 0.12370E-07),
    12: ("K-", 4, 0.4937E+00, -1., 0.12370E-07),
    13: ("neutron", 3, 0.9396E+00, 0., 0.88700E+03),
    14: ("proton", 4, 0.9383E+00, 1., 0.10000E+16),
    15: ("anti-proton", 4, 0.9383E+00, -1., 0.10000E+16),
    16: ("K0S", 3, 0.4977E+00, 0., 0.89260E-10),
    17: ("eta", 3, 0.5475E+00, 0., 0.54850E-18),
    18: ("lambda", 3, 0.1116E+01, 0., 0.26320E-09),
    19: ("sigma+", 4, 0.1189E+01, 1., 0.79900E-10),
    20: ("sigma0", 3, 0.1193E+01, 0., 0.74000E-19),
    21: ("sigma-", 4, 0.1197E+01, -1., 0.14790E-09),
    22: ("xi0", 3, 0.1315E+01, 0., 0.29000E-09),
    23: ("xi-", 4, 0.1321E+01, -1., 0.16390E-09),
    24: ("omega-", 4, 0.1672E+01, -1., 0.82200E-10),
    25: ("anti-neutron", 3, 0.9396E+00, 0., 0.88700E+03),
    26: ("anti-lambda", 3, 0.1116E+01, 0., 0.26320E-09),
    27: ("anti-sigma-", 4, 0.1189E+01, -1., 0.79900E-10),
    28: ("anti-sigma0", 3, 0.1193E+01, 0., 0.74000E-19),
    29: ("anti-sigma+", 4, 0.1197E+01, 1., 0.14790E-09),
    30: ("anti-xi0", 3, 0.1315E+01, 0., 0.29000E-09),
    31: ("anti-xi+", 4, 0.1321E+01, 1., 0.16390E-09),
    32: ("anti-omega+", 4, 0.1672E+01, 1., 0.82200E-10),
    45: ("deuteron", 8, 0.1876E+01, 1., 0.10000E+16),
    46: ("triton", 8, 0.2809E+01, 1., 0.10000E+16),
    47: ("alpha", 8, 0.3727E+01, 2., 0.10000E+16),
    48: ("geantino", 6, 0.0000E+00, 0., 0.10000E+16),
    49: ("He3", 8, 0.2809E+01, 2., 0.10000E+16),
    50: ("Cerenkov", 7, 0.0000E+00, 0., 0.10000E+16)
}


# from data_files/SomeMiniBooNEDetails_on_files_v2.pdf
nu_types = {
    1: "numu",
    2: "numubar",
    3: "nue",
    4: "nuebar"
}

# https://www.sciencedirect.com/science/article/pii/S0168900208015404
index_of_refraction = 1.47

def get_particle_name(particle_code):
    return particles[particle_code][0]

def get_cherenkov_threshold(particle_code):
    charge = particles[particle_code][3]
    if charge == 0:
        return np.nan
    if particle_code == 1 or particle_code == 2 or particle_code == 3:
        return np.nan # shower, different type of energy threshold, not considered here
    
    particle_mass = particles[particle_code][2]

    beta_threshold = 1 / index_of_refraction
    gamma_threshold = 1 / np.sqrt(1 - beta_threshold**2)

    kinetic_energy_threshold = (gamma_threshold - 1) * particle_mass * 1000 # MeV

    return kinetic_energy_threshold
    


In [ ]:
print("particle: cherenkov threshold\n")
for particle_code in particles:
    print(f"{get_particle_name(particle_code)}: {get_cherenkov_threshold(particle_code)}")


In [ ]:
max_file_to_load = 1

shower_low_threshold = 10
shower_high_threshold = 30

one_HE_shower_0_HE_track_df = pd.DataFrame()
two_HE_shower_0_HE_track_df = pd.DataFrame()

for file_i in tqdm(range(1, max_file_to_load + 1)):

    f = uproot.open(f"data_files/output_osc_mc_detail_{file_i}.root")

    df = f["MiniBooNE_CCQE"].arrays(["EventNumber", "Weight", "PassOsc", "RecoEnuQE", "Energy", "CosTheta", "TrueEnergy", "NuType", "NFSP", "FSPType", "MomX", "MomY", "MomZ", "MomT"], library="pd")

    df["nu_type"] = df["NuType"].apply(lambda x: nu_types[x])
    df.drop(columns=["NuType"], inplace=True)

    particle_codes = df["FSPType"].to_numpy()
    particle_MomXs = df["MomX"].to_numpy()
    particle_MomYs = df["MomY"].to_numpy()
    particle_MomZs = df["MomZ"].to_numpy()
    particle_MomTs = df["MomT"].to_numpy()

    num_true_HE_showers = []
    num_true_ME_showers = []
    num_true_HE_tracks = []
    for i in range(len(particle_codes)):
        curr_num_HE_showers = 0
        curr_num_ME_showers = 0
        curr_num_HE_tracks = 0
        curr_particle_codes = particle_codes[i]
        curr_MomTs = particle_MomTs[i]
        for j in range(len(curr_particle_codes)):
            curr_particle_name = get_particle_name(curr_particle_codes[j])
            curr_particle_energy = curr_MomTs[j] * 1000

            if (curr_particle_name == "gamma" or curr_particle_name == "electron" or curr_particle_name == "positron"):
                if shower_low_threshold <= curr_particle_energy <= shower_high_threshold:
                    curr_num_ME_showers += 1
                if shower_high_threshold <= curr_particle_energy:
                    curr_num_HE_showers += 1

            else:
                cherenkov_threshold = get_cherenkov_threshold(curr_particle_codes[j])
                if cherenkov_threshold is not np.nan and curr_particle_energy > cherenkov_threshold:
                    curr_num_HE_tracks += 1

        num_true_ME_showers.append(curr_num_ME_showers)
        num_true_HE_showers.append(curr_num_HE_showers)
        num_true_HE_tracks.append(curr_num_HE_tracks)

    df["num_true_ME_showers"] = num_true_ME_showers
    df["num_true_HE_showers"] = num_true_HE_showers
    df["num_true_HE_tracks"] = num_true_HE_tracks

    curr_two_HE_shower_0_HE_track_df = df.query("num_true_HE_showers == 2 and num_true_ME_showers == 0 and num_true_HE_tracks == 0")
    curr_one_HE_shower_0_HE_track_df = df.query("num_true_HE_showers == 1 and num_true_ME_showers == 0 and num_true_HE_tracks == 0")

    two_HE_shower_0_HE_track_df = pd.concat([two_HE_shower_0_HE_track_df, curr_two_HE_shower_0_HE_track_df])
    one_HE_shower_0_HE_track_df = pd.concat([one_HE_shower_0_HE_track_df, curr_one_HE_shower_0_HE_track_df])


In [ ]:
two_HE_shower_0_HE_track_df


In [ ]:
one_HE_shower_0_HE_track_df


# True Quantities for two showers

In [ ]:
particle_codes = two_HE_shower_0_HE_track_df["FSPType"].to_numpy()
particle_MomXs = two_HE_shower_0_HE_track_df["MomX"].to_numpy()
particle_MomYs = two_HE_shower_0_HE_track_df["MomY"].to_numpy()
particle_MomZs = two_HE_shower_0_HE_track_df["MomZ"].to_numpy()
particle_MomTs = two_HE_shower_0_HE_track_df["MomT"].to_numpy()

leading_energies = []
subleading_energies = []
opening_angles = []
total_momentum_beam_angles = []

for i in tqdm(range(len(particle_codes))):

    curr_leading_energy = -np.inf
    curr_subleading_energy = -np.inf
    curr_leading_momentum = np.array([np.nan, np.nan, np.nan])
    curr_subleading_momentum = np.array([np.nan, np.nan, np.nan])

    curr_particle_codes = particle_codes[i]
    curr_MomXs = particle_MomXs[i]
    curr_MomYs = particle_MomYs[i]
    curr_MomZs = particle_MomZs[i]
    curr_MomTs = particle_MomTs[i]

    for j in range(len(curr_particle_codes)):
        curr_particle_name = get_particle_name(curr_particle_codes[j])
        curr_particle_energy = curr_MomTs[j] * 1000
        curr_particle_momentum = np.array([curr_MomXs[j], curr_MomYs[j], curr_MomZs[j]])
        if curr_particle_name == "gamma" or curr_particle_name == "electron" or curr_particle_name == "positron" and curr_particle_energy > 20:
            if curr_particle_energy > curr_leading_energy: # new highest energy shower
                curr_subleading_energy = curr_leading_energy
                curr_subleading_momentum = curr_leading_momentum
                curr_leading_energy = curr_particle_energy
                curr_leading_momentum = curr_particle_momentum
            elif curr_particle_energy > curr_subleading_energy: # new subleading shower
                curr_subleading_energy = curr_particle_energy
                curr_subleading_momentum = curr_particle_momentum
        
    curr_total_momentum = curr_leading_momentum + curr_subleading_momentum
    curr_total_momentum_magnitude = np.linalg.norm(curr_total_momentum)
    curr_total_momentum_beam_angle = np.arccos(curr_total_momentum[2] / curr_total_momentum_magnitude) * 180 / np.pi

    curr_opening_angle = np.arccos(np.dot(curr_leading_momentum, curr_subleading_momentum) / (np.linalg.norm(curr_leading_momentum) * np.linalg.norm(curr_subleading_momentum))) * 180 / np.pi

    leading_energies.append(curr_leading_energy)
    subleading_energies.append(curr_subleading_energy)
    opening_angles.append(curr_opening_angle)
    total_momentum_beam_angles.append(curr_total_momentum_beam_angle)

two_HE_shower_0_HE_track_df["leading_energy"] = leading_energies
two_HE_shower_0_HE_track_df["subleading_energy"] = subleading_energies
two_HE_shower_0_HE_track_df["opening_angle"] = opening_angles
two_HE_shower_0_HE_track_df["total_momentum_beam_angle"] = total_momentum_beam_angles

two_HE_shower_0_HE_track_df["energy_asymmetry"] = two_HE_shower_0_HE_track_df["leading_energy"] / (two_HE_shower_0_HE_track_df["leading_energy"] + two_HE_shower_0_HE_track_df["subleading_energy"])
two_HE_shower_0_HE_track_df["invariant_mass"] = np.sqrt(2 * two_HE_shower_0_HE_track_df["leading_energy"] * two_HE_shower_0_HE_track_df["subleading_energy"] * (1 - np.cos(np.pi / 180 * two_HE_shower_0_HE_track_df["opening_angle"])))


In [ ]:
passing_two_HE_shower_0_HE_track_df = two_HE_shower_0_HE_track_df.query("PassOsc == True")
passing_one_HE_shower_0_HE_track_df = one_HE_shower_0_HE_track_df.query("PassOsc == True")


# Distributions

In [ ]:
plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(10, 7))
plt.scatter(two_HE_shower_0_HE_track_df["leading_energy"], two_HE_shower_0_HE_track_df["subleading_energy"], s=0.1, c="k", label="All Events")
plt.scatter(passing_two_HE_shower_0_HE_track_df["leading_energy"], passing_two_HE_shower_0_HE_track_df["subleading_energy"], s=5, c="r", label="Passing Events")
plt.xlabel("Leading Shower Energy (MeV)")
plt.ylabel("Subleading Shower Energy (MeV)")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_high_threshold} MeV Showers\nZero {shower_low_threshold}-{shower_high_threshold} MeV Showers\nZero Tracks above Cherenkov Threshold")
plt.xlim(0, 1000)
plt.ylim(0, 500)
plt.legend()
plt.show()

plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(10, 7))
plt.scatter(two_HE_shower_0_HE_track_df["leading_energy"], two_HE_shower_0_HE_track_df["opening_angle"], s=0.1, c="k", label="All Events")
plt.scatter(passing_two_HE_shower_0_HE_track_df["leading_energy"], passing_two_HE_shower_0_HE_track_df["opening_angle"], s=5, c="r", label="Passing Events")
plt.xlabel("Leading Shower Energy (MeV)")
plt.ylabel("True Opening Angle (degrees)")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_high_threshold} MeV Showers\nZero {shower_low_threshold}-{shower_high_threshold} MeV Showers\nZero Tracks above Cherenkov Threshold")
plt.xlim(0, 1000)
plt.ylim(0, 180)
plt.legend()
plt.show()

plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(10, 7))
plt.scatter(two_HE_shower_0_HE_track_df["leading_energy"], two_HE_shower_0_HE_track_df["invariant_mass"], s=0.1, c="k", label="All Events")
plt.scatter(passing_two_HE_shower_0_HE_track_df["leading_energy"], passing_two_HE_shower_0_HE_track_df["invariant_mass"], s=5, c="r", label="Passing Events")
plt.xlabel("Leading Shower Energy (MeV)")
plt.ylabel("True Invariant Mass (MeV)")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_high_threshold} MeV Showers\nZero {shower_low_threshold}-{shower_high_threshold} MeV Showers\nZero Tracks above Cherenkov Threshold")
#plt.xlim(0, 1000)
#plt.ylim(0, 1000)
plt.legend()
plt.show()



In [ ]:
print(1/0)

# Efficiencies

In [ ]:
plt.rcParams.update({'font.size': 16})

#opening_angle_bins = np.linspace(0, 180, 18+1)
#opening_angle_bins = np.linspace(0, 90, 18+1)
opening_angle_bins = np.linspace(0, 45, 16)
opening_angle_bin_centers = [(opening_angle_bins[i] + opening_angle_bins[i+1]) / 2 for i in range(len(opening_angle_bins) - 1)]

total_counts = np.histogram(two_HE_shower_0_HE_track_df["opening_angle"], weights=two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins)[0]
passing_counts = np.histogram(passing_two_HE_shower_0_HE_track_df["opening_angle"], weights=passing_two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins)[0]
effs = passing_counts / total_counts

eff_errs = np.sqrt(effs * (1 - effs) / total_counts)

plt.figure(figsize=(10, 7))
plt.hist(two_HE_shower_0_HE_track_df["opening_angle"], weights=two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", label="All Events")
plt.hist(passing_two_HE_shower_0_HE_track_df["opening_angle"], weights=passing_two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", label="Passing Events")
plt.xlabel("True Opening Angle (degrees)")
plt.ylabel("Events")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_high_threshold} MeV Showers\nZero {shower_low_threshold}-{shower_high_threshold} MeV Showers\nZero Tracks above Cherenkov Threshold")
plt.legend()
plt.yscale("log")
plt.xlim(opening_angle_bins[0], opening_angle_bins[-1])
plt.show()

plt.figure(figsize=(10, 7))
plt.hist(two_HE_shower_0_HE_track_df["opening_angle"], weights=two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", density=True, label="All Events")
plt.hist(passing_two_HE_shower_0_HE_track_df["opening_angle"], weights=passing_two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", density=True, label="Passing Events")
plt.xlabel("True Opening Angle (degrees)")
plt.ylabel("Events (Normalized)")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_high_threshold} MeV Showers\nZero {shower_low_threshold}-{shower_high_threshold} MeV Showers\nZero Tracks above Cherenkov Threshold")
plt.legend()
plt.xlim(opening_angle_bins[0], opening_angle_bins[-1])
plt.show()

plt.figure(figsize=(10, 7))
plt.errorbar(opening_angle_bin_centers, effs, yerr=eff_errs, fmt="o", capsize=5)
plt.xlabel("True Opening Angle (degrees)")
plt.ylabel("MiniBooNE nue CCQE Selection Efficiency")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_high_threshold} MeV Showers\nZero {shower_low_threshold}-{shower_high_threshold} MeV Showers\nZero Tracks above Cherenkov Threshold")
plt.ylim(0, 0.1)
plt.xlim(opening_angle_bins[0], opening_angle_bins[-1])
plt.show()